In [4]:
from ecell4 import *
%matplotlib inline
import numpy as np
import math
import sys
from math import sqrt
import matplotlib.pyplot as plt
from ecell4.extra import ensemble
import numpy as np
from math import pi

In [5]:
def wrap(job,job_id,task_id):
    def singlerun(kf,K,correct,NA,NB,L,run,durfac):
        import numpy as np
        rng = GSLRandomNumberGenerator()
        rng.seed(run)
        rm = 0.005
        factor = (2 * sqrt(2.0) + 4 * sqrt(3.0) + 3 * sqrt(6.0) + sqrt(22.0))**2/(72 * (6 * sqrt(2.0) + 4 * sqrt(3.0) + 3 * sqrt(6.0)))
        R=2*rm
        voxelr=rm
        D = 1.
        ka=kf*2*pi*D
        gamma=0.577
        td =(2*voxelr)**2/(4*D)
        duration=2#td*10**durfac
        H = 3*voxelr*2
        kd=ka/K        
        if correct==True:
            PP=1./(1+math.sqrt(3)*(math.log(1./12)+(4*math.pi*D/ka)-2*gamma)/2/math.pi)
            ka = PP*D/factor
        ori = Real3(0.5*H,0,0)
        unit0 = Real3(0,0,L)
        unit1 = Real3(0,L,0)        
        w = spatiocyte.SpatiocyteWorld(Real3(H,L,L),voxelr,rng)
        #size=tuple(w.calculate_shape(Real3(H,L,L),voxelr))
        #Nmv=size[1]*size[2]    
        #print('phil=',N/Nmv)
        lx,ly,lz=w.actual_lengths()
        Area= ly*lz
        #Area=Nmv*(2*voxelr)**2
        c0=NB/Area
        sinf=kd/(kd+c0*ka)
        #print(ka,kd)
        with species_attributes():
            A |  {'D': str(0), 'radius':str(voxelr),'location':'X'}
            B |  {'D': str(D), 'radius':str(voxelr),'location':'X'}
            C |  {'D': str(0), 'radius':str(voxelr),'location':'X'}

        with reaction_rules():#for spa and egfrd
            A + B == C | (ka,kd)

        m2 = get_model()
        w.bind_to(m2)
        w.add_structure(Species('X'),PlanarSurface(ori,unit0,unit1))
        w.add_molecules(Species('A'), NA)
        w.add_molecules(Species('B'), NB)
        sim = spatiocyte.SpatiocyteSimulator(w)
        sim.initialize()
        obs=FixedIntervalNumberObserver(td,'A')
        sim.run(duration,obs)
        out=np.array(obs.data()).T
        if correct==True:
            #name = '/home/chew/outputdata/2DRD/newP/kfac'+str(kaf)+'/run'+str(run)
            name = '/home/chew/outputdata/2DRD/newP/kf'+str(kf)+'K'+str(K)+'/run'+str(run)
        else:
            #name = '/home/chew/outputdata/2DRD/oriP/kfac'+str(kaf)+'/run'+str(run)
            name = '/home/chew/outputdata/2DRD/oriP/kf'+str(kf)+'K'+str(K)+'/run'+str(run)
        #filename=open(name,'w')
        #np.savetxt(name,np.column_stack((out[0],out[1])),delimiter=',',fmt='%s')
        #filename.close()          
        return 
    job.update({'run':task_id})
    out=singlerun(**job)
    return 

NB=500
NA=int(0.05*NB)
N=NA+NB
rm = 0.005
phi=0.1/100
L=rm*sqrt(N*pi/phi)
print(NA,NB,N,L)
assert(NA>1)
K=1#ka/kd
durfac=1#4.5
simrun=1#300#000

for probdef in [True]:#,False]:#, True]:
    for kf in [0.1,10]:#,500]:#,10]:
        jobs=[{'durfac':durfac,'L':L,'kf':kf,'K':K,'correct':probdef,'NA':NA,'NB':NB}]
        res=ensemble.run_slurm(wrap,jobs,n=simrun,nproc=350,path='tmp',modules=('numpy','ecell4','math'))[0]
        #res=ensemble.run_multiprocessing(wrap,jobs,n=simrun,nproc=60,modules=('numpy','ecell4','math'))[0]
print('finish') 

DEBUG:sge:Submitted batch job 1225444
DEBUG:sge:             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
DEBUG:sge:   1225444_[1%350]      defq sge-tgcv     chew PD       0:00      1 (None)
DEBUG:sge:
INFO:sge:Job 1225444 must be queued, running or being transferred


25 500 525 6.421324129676528


INFO:sge:Waiting for jobids [1225444] to finish
DEBUG:sge:             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
DEBUG:sge:         1225444_1      defq sge-tgcv     chew  R       0:10      1 ibc01
DEBUG:sge:
INFO:sge:Job 1225444 must be queued, running or being transferred
INFO:sge:Waiting for jobids [1225444] to finish
DEBUG:sge:             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
DEBUG:sge:         1225444_1      defq sge-tgcv     chew  R       0:20      1 ibc01
DEBUG:sge:
INFO:sge:Job 1225444 must be queued, running or being transferred
INFO:sge:Waiting for jobids [1225444] to finish
DEBUG:sge:             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
DEBUG:sge:         1225444_1      defq sge-tgcv     chew  R       0:30      1 ibc01
DEBUG:sge:
INFO:sge:Job 1225444 must be queued, running or being transferred
INFO:sge:Waiting for jobids [1225444] to finish
DEBUG:sge:             JOBID PARTITION  

KeyboardInterrupt: 

In [3]:
plt.semilogx(res[0][0],res[0][1]/NA)


NameError: name 'plt' is not defined

In [9]:
voxelr=0.005
H = 3*voxelr*2
w = spatiocyte.SpatiocyteWorld(Real3(H,L,L),voxelr)
size=tuple(w.calculate_shape(Real3(H,L,L),voxelr))
Nmv=size[1]*size[2]    
print('phil={}%'.format(N*100/Nmv))
Area=Nmv*(2*voxelr)**2
c0=NB/Area
print('c0',c0)

phil=0.10986757295206843%
c0 10.463578376387469


In [10]:
c0=10.4636
kf=0.1
ka=kf*2*pi*1
K=1
kd=ka/K
sinf=kd/(kd+c0*ka)
ka,kd

(0.6283185307179586, 0.6283185307179586)

In [54]:
(2*voxelr)**2/(4*1)

2.5e-05

In [56]:
1.25/5

0.25